In [1]:
import warnings
warnings.filterwarnings('ignore')

from sqlalchemy import (create_engine, Table, Column, Integer, String, MetaData, inspect)
import pandas as pd
import pymysql.cursors
import matplotlib.pyplot as plt
import numpy as np
import math
import nltk

In [2]:
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='root',#put your password for mysql server here.
                             db='yelp_db',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

raw_data = pd.read_sql('''
select business.id,
  business.name,
  business.latitude,
  business.longitude,
  business.neighborhood,
  category.category
  FROM category
  inner join business on
  business.id = category.business_id
  where business.city ="Pittsburgh"
  and category.category = "Italian"
  or category.category = "Chinese"
  or category.category = "Mexican"
  or category.category = "American (New)"
  or category.category = "Mediterranean"
  or category.category = "Japanese"
  or category.category = "Thai"
  or category.category = "Indian"
  or category.category = "Middle Eastern"
  or category.category = "Greek"
  or category.category = "French"
  or category.category = "Korean"
  or category.category = "Turkish"
  or category.category = "Caribbean"
  or category.category = "Vietnamese"
  or category.category = "Latin American"
  or category.category = "Taiwanese"
  GROUP BY business.id, category.category;''',connection) #.values

In [ ]:
from sklearn.preprocessing import LabelEncoder

# encodes any df column of type category
def encoder(df):
        columnsToEncode = list(df.select_dtypes(include=['category','object']))
        le = LabelEncoder()
        for feature in columnsToEncode:
            try:
                df[feature] = le.fit_transform(df[feature])
            except:
                print('Error encoding '+ feature)
        return df
    
from sklearn.cluster import KMeans
features = ['latitude', 'longitude', 'category']
pitts_cuisines = raw_data[features]
sub_data = encoder(pitts_cuisines)
#print(sub_data.head())
kmeans = KMeans(n_clusters=17, random_state=0).fit(sub_data)

In [ ]:
# optimal number of clusters
from sklearn.metrics import silhouette_score
silhouette_score(sub_data, kmeans.labels_)

In [ ]:
# visualize the clusers
from sklearn import decomposition
pca = decomposition.PCA(n_components=10)
pca.fit(sub_data)
X = pca.transform(sub_data)
X.flatten()

import matplotlib.pyplot as plt
plt.plot(X.flatten(), 'x')
plt.show()